In [ ]:
! pip install markupsafe

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

from ner.evaluating_confidence import load_generated_data_for_confidence


In [6]:
all_data[0]

{'data_point_idx': '0',
 'spans': [['JAPAN', 'LOC'], ['CHINA', 'PER']],
 'logits_for_tags': [{'entity': ('Japan', 'LOC'),
   'gold tag': 'LOC',
   'outputted_tag': 'LOC',
   'tags_logits': {'PER': 14.58165168762207,
    'ORG': 12.636054992675781,
    'LOC': 16.61681365966797,
    'MISC': 12.887547492980957,
    'None': 13.413434028625488},
   'confidence': {'PER': 0.10763013730626471,
    'ORG': 0.015380554243304057,
    'LOC': 0.8237463600457737,
    'MISC': 0.019778520000346703,
    'None': 0.03346442840431089}},
  {'entity': ('China', 'LOC'),
   'gold tag': 'PER',
   'outputted_tag': 'LOC',
   'tags_logits': {'PER': 14.925336837768555,
    'ORG': 14.356271743774414,
    'LOC': 19.757312774658203,
    'MISC': 16.524211883544922,
    'None': 13.688724517822266},
   'confidence': {'PER': 0.0075607221377126735,
    'ORG': 0.004279780026078898,
    'LOC': 0.9485577217806506,
    'MISC': 0.03740639773662025,
    'None': 0.002195378318937564}}]}

In [8]:
all_data = load_generated_data_for_confidence()
output = []
for dp in all_data:
    for ent in dp['logits_for_tags']:
        output.append(list(ent['tags_logits'].values())+[ent['gol']])

In [9]:
output[0]

[14.58165168762207,
 12.636054992675781,
 16.61681365966797,
 12.887547492980957,
 13.413434028625488,
 {'PER': 14.58165168762207,
  'ORG': 12.636054992675781,
  'LOC': 16.61681365966797,
  'MISC': 12.887547492980957,
  'None': 13.413434028625488}]

In [ ]:

# Load your dataset
# df = pd.read_csv('your_dataset.csv')

# Assuming you have a dataframe 'df' with columns: feature_1, feature_2, feature_3, feature_4, feature_5, label

# Split the data into features (X) and labels (y)
X = df[['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5']]
y = df['label']

# Convert labels to categorical (one-hot encoding)
y_categorical = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(5,)))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_classes = [int(i) for i in y_pred.argmax(axis=-1)]
y_test_classes = [int(i) for i in y_test.argmax(axis=-1)]

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test accuracy: {accuracy}')
